#### Load HumanEval and generate perturbations in solution code

In [10]:
import ast
import random
import pandas as pd
import difflib

from utils import Perturber

In [12]:
import pandas as pd
bugfactory = Perturber()

def perturb_humaneval(filepath):
    df = pd.read_csv(filepath)
    df['solution'] = df['prompt'] + df['canonical_solution']

    var_perturbations, expr_perturbations = list(), list()
    for i, row in df.iterrows():
        code = row['solution']
        var_bug = bugfactory.randomly_modify_variable(code)
        if not bugfactory.same_tree(code, var_bug):
            var_perturbations.append(var_bug)
        
        expr_bug = bugfactory.randomly_modify_expression(code)
        if not bugfactory.same_tree(code, expr_bug):
            expr_perturbations.append(expr_bug)
    return list(df['solution']), var_perturbations, expr_perturbations
        
solution, variable_changes, expression_changes = perturb_humaneval('data/test.csv')

#### Check out results 😄

In [13]:
print(f'Dataset size: {len(solution)}')
print(f'Successful variable perturbations: {len(variable_changes)}')
print(f'Successful expression perturbations: {len(expression_changes)}')

Dataset size: 164
Successful variable perturbations: 162
Successful expression perturbations: 122


This project is still in its weekend-hack stages, the perturbations for all possible expression types and variable types have not been implemented yet. In addition, if a particular Python function does not have more than one variable, we **cannot** randomly sample a replacement to reinsert in the code. Excited to dig deeper into ast hierarchical structure and add more possibilities soon.

Note the difference in the last line of the the two outputs below; the original solution calculates the mean absolute deviation by dividing the sum of deviations by the size of the number list, while the corrupted solution divides by the SUM of the number list.

In [19]:
print(solution[4])

from typing import List


def mean_absolute_deviation(numbers: List[float]) -> float:
    """ For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0
    """
    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)



In [20]:
print(variable_changes[4])

from typing import List

def mean_absolute_deviation(numbers: List[float]) -> float:
    """ For a given list of input numbers, calculate Mean Absolute Deviation
    around the mean of this dataset.
    Mean Absolute Deviation is the average absolute difference between each
    element and a centerpoint (mean in this case):
    MAD = average | x - x_mean |
    >>> mean_absolute_deviation([1.0, 2.0, 3.0, 4.0])
    1.0
    """
    mean = sum(numbers) / len(numbers)
    return sum((abs(x - mean) for x in numbers)) / sum(numbers)
